In [1]:
import os

In [2]:
%pwd

'd:\\Projects\\Chicken Disease Classification\\research'

In [3]:
os.chdir("../")


In [4]:
%pwd

'd:\\Projects\\Chicken Disease Classification'

In [29]:
import tensorflow as tf
model = tf.keras.models.load_model("artifacts/training/model.h5")


In [30]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [31]:
from CDC.constants import *
from CDC.utils.helper_functions import read_yaml, create_directories, save_json

In [32]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model=Path("artifacts/training/model.h5"),
            training_data=Path("artifacts/data/Chicken-fecal-images"),
            all_params=self.params,
            params_image_size=self.params.INPUT_SHAPE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [46]:
from urllib.parse import urlparse
import json

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), content=scores)



In [47]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    

except Exception as e:
   raise e

[2023-06-21 22:54:22,736: INFO: helper_functions: yaml file config\config.yaml uploaded successfully]
[2023-06-21 22:54:22,743: INFO: helper_functions: yaml file params.yaml uploaded successfully]
[2023-06-21 22:54:22,746: INFO: helper_functions: created directory at: artifacts]
Found 116 images belonging to 2 classes.
8/8 [==============================] - 48s 6s/step - loss: 1.6698 - accuracy: 0.8276


In [42]:
import json
save_json(path=Path("/file.json"), content={'key':'val'})

NameError: name 'json' is not defined

In [45]:
from CDC.utils.helper_functions import save_json
save_json(path=Path("/file.txt"), content={'key':'val'})

NameError: name 'json' is not defined